# YANK Simulation Health Report

General Settings
========

Manditory Settings
----------------
* `store_directory`: Location where the experiment was run. This has an `analysis.yaml` file and two `.nc` files.

Optional Settings
----------------
* `decorrelation_threshold`: When number of decorrelated samples is less than this percent of the total number of samples, raise a warninig. Default: `0.1`.
* `mixing_cutoff`: Minimal level of mixing percent from state `i` to `j` that will be plotted. Default: `0.05`.
* `mixing_warning_threshold`: Level of mixing where transition from state `i` to `j` generates a warning based on percent of total swaps. Default: `0.90`.
* `phase_stacked_replica_plots`: Boolean to set if the two phases' replica mixing plots should be stacked one on top of the other or side by side. If `True`, every replica will span the whole notebook, but the notebook will be longer. If `False`, the two phases' plots will be next to each other for a shorter notebook, but a more compressed view. Default `False`.

In [ ]:
# Manditory Settings
store_directory = 'STOREDIRBLANK'

# Optional Settings
decorrelation_threshold = 0.1 # Percentage on which warrning is raised
mixing_cutoff = 0.05
mixing_warning_threshold = 0.90
phase_stacked_replica_plots = False

## Data Imports

These are the imports and files which will be referenced for the report

In [ ]:
import os
import yaml
import netCDF4 as nc
import numpy as np
from pymbar import timeseries, MBAR
import yank.analyze as analyze
%matplotlib inline
from matplotlib import pyplot as plt

# Read in data
analysis_script_path = os.path.join(store_directory, 'analysis.yaml')
if not os.path.isfile(analysis_script_path):
    err_msg = 'Cannot find analysis.yaml script in {}'.format(store_directory)
    raise RuntimeError(err_msg)
with open(analysis_script_path, 'r') as f:
    analysis = yaml.load(f)
phases = []
signs = {}
ncfiles = {}
for phase, sign in analysis:
    phases.append(phase)
    signs[phase] = sign
    ncfile_path = os.path.join(store_directory, phase + '.nc')
    ncfiles[phase] = nc.Dataset(ncfile_path, 'r')
lenphases = len(phases)

## General Simulation Data:

In [ ]:
iterations = {}
nstates = {}
natoms = {}
for phase in phases:
    positions = ncfiles[phase].variables['positions']
    iterations[phase], nstates[phase], natoms[phase], spatial = positions.shape

leniter = max(len('Iterations'), *[len(str(i)) for i in iterations.values()]) + 2
lenstates = max(len('States'), *[len(str(i)) for i in nstates.values()]) + 2
lennatoms = max(len('Num Atoms'), *[len(str(i)) for i in natoms.values()]) + 2
lenleftcol = max(len('Phase'), *[len(phase) for phase in phases]) + 2

lines = []
headstring = ''
headstring += ('{:^' + '{}'.format(lenleftcol) + '}').format('Phase') + '|'
headstring += ('{:^' + '{}'.format(leniter) + '}').format('Iterations') + '|'
headstring += ('{:^' + '{}'.format(lenstates) + '}').format('States') + '|'
headstring += ('{:^' + '{}'.format(lennatoms) + '}').format('Num Atoms')
lines.append(headstring)
lenline = len(headstring)
topdiv = '='*lenline
lines.append(topdiv)
for phase in phases:
    phasestring = ''
    phasestring += ('{:^' + '{}'.format(lenleftcol) + '}').format(phase) + '|'
    phasestring += ('{:^' + '{}'.format(leniter) + '}').format(iterations[phase]) + '|'
    phasestring += ('{:^' + '{}'.format(lenstates) + '}').format(nstates[phase]) + '|'
    phasestring += ('{:^' + '{}'.format(lennatoms) + '}').format(natoms[phase])
    lines.append(phasestring)
    lines.append('-'*lenline)

for line in lines:
    print(line)

Equilibration
=============

How to interpret these plots
--------------------------
Shown is the potential energy added up accross all replicas (black dots), the moving average (red line), and where we have auto-detected the equlibration (blue line) for each phase. Finally, the total number of decorrelated samples for each phase is attached to each plot.

You want to see a majority of samples to the right of the blue line and the red line converging to a constant value.  If you do not see these trends or you think there are insufficient samples, please consider running for longer. 

For additional information on the theory of these charts, please see the Equilibration Primer at the Appendix of the report

In [ ]:
u_ns = {}
nequils = {}
g_ts = {}
Neff_maxs = {}

from scipy import interpolate
plt.rcParams['figure.figsize'] = 20, 6*lenphases 
plt.clf()
f = plt.gcf()
f.subplots_adjust(hspace=0.4)
plotkeys = [100*lenphases + 10 + (i+1) for i in range(lenphases)]
for phase, plotid in zip(phases, plotkeys):
    p = f.add_subplot(plotid)
    u_ns[phase] = analyze.extract_u_n(ncfiles[phase])
    u_ns[phase] = u_ns[phase][1:] # Correction for bug
    nequils[phase], g_ts[phase], Neff_maxs[phase] = timeseries.detectEquilibration(u_ns[phase])
    y = u_ns[phase]
    N = y.size
    x = np.arange(N)
    # Scatter
    p.plot(x, y, 'k.')
    # Smoothed equalibrium
    tck = interpolate.splrep(x, y, k = 5, s = N*1E7)
    smoothed = interpolate.splev(x, tck, der = 0)
    p.plot(x, smoothed, '-r', linewidth=4)
    # Nequil line
    ylim = p.get_ylim()
    p.vlines(nequils[phase], *ylim, colors='b', linewidth=4)
    p.set_ylim(*ylim) # Reset limits incase vlines expanded them
    p.set_xlim([0,N])
    # Set text
    p.set_title(phase + " phase", fontsize=20)
    p.set_ylabel(r'$\Sigma_n u_n$ in kT', fontsize=20)
    p.set_xlabel('Iteration', fontsize=20)
    # Extra info in text boxes
    subsample_string = 'Subsample Rate: {0:.2f}\nDecorelated Samples: {1:d}'.format(g_ts[phase], int(np.floor(Neff_maxs[phase])))
    if np.mean([0,N]) > nequils[phase]:
        txt_horz = 'right'
        txt_xcoord = 0.95
    else:
        txt_horz = 'left'
        txt_xcoord = 0.05
    smooth_index = {'right':-1, 'left':0} # condition y
    if np.mean(ylim) > smoothed[smooth_index[txt_horz]]:
        txt_vert = 'top'
        txt_ycoord = 0.95
    else:
        txt_vert = 'bottom'
        txt_ycoord = 0.05
    p.text(txt_xcoord, txt_ycoord,
           subsample_string,
           verticalalignment=txt_vert, horizontalalignment=txt_horz,
           transform=p.transAxes,
           fontsize=15,
           bbox={'alpha':1.0, 'facecolor':'white'}
          )
plt.show()

Additional Decorrelation Analysis
==================

The following Pie Charts show you the breakdown of how many samples were kept, and how many were lost to either equilibration or decorrelation. Warnings are shown when below a threshold (originally written to be 10%)

In [ ]:
plt.rcParams['figure.figsize'] = 20, 8
plt.clf()
f = plt.gcf()
f.subplots_adjust(wspace=0.2)
plotkeys = [100 + (10*lenphases) + (i+1) for i in range(lenphases)] # Horizonal distribution
for phase, plotid in zip(phases, plotkeys):
    p = f.add_subplot(plotid)
    N = iterations[phase]
    labels = ['Decorrelated', 'Correlated', 'Equilibration']
    colors = ['#2c7bb6', '#abd0e0', '#fdae61'] # blue, light blue, and orange
    explode = [0, 0, 0.0]
    eq = nequils[phase]
    decor = int(np.floor(Neff_maxs[phase]))
    cor = N - eq - decor
    dat = np.array([decor, cor, eq])/float(N)
    if dat[0] <= decorrelation_threshold:
        colors[0] = '#d7191c' # Red for warning
    patch, txt, autotxt = p.pie(
         dat, 
         explode=explode, 
         labels=labels, 
         colors = colors,
         autopct='%1.1f%%',
         shadow=True, 
         startangle=90 + 360*dat[0]/2, # put center of decor at top
         counterclock=False,
         textprops={'fontsize':14}
         )
    for tx in txt: # This is the only way I have found to adjust the label font size
        tx.set_fontsize(18)
    p.axis('equal')
    p.set_title(phase + " phase", fontsize=20, y=1.05)
    if dat[0] <= decorrelation_threshold:
        p.text(
            0.5, -0.1,
            "Warning! Fewer than {0:.1f}% samples are\nequilibrated and decorelated!".format(decorrelation_threshold*100),
            verticalalignment='bottom', horizontalalignment='center',
            transform=p.transAxes,
            fontsize=20,
            color='red',
            bbox={'alpha':1.0, 'facecolor':'white', 'lw':0, 'pad':0}
            )
plt.show()

Mixing statistics
=================

We can analyze the "mixing statistics" of the equilibrated part of the simulation to ensure that the $(X,S)$ chain is mixing reasonably well among the various alchemical states.

For information on how this is computed, including how to interpret the Perron Eigenvalue, please see the *Mixing Statistics Primer* at the end of the report.


What do you want to see?
-----------------------
You want a replica to mix into other replicas, so you want a diffusion of configurations shown by a spread out color map in the figure. What you don't want to see is highly concentrated replicas that do not mix at all. The graphs will show red and generate a warning if there are replicas that do not mix well.

For the Perron/subdominant eigenvalue, you want to see a value smaller than one `1`. The further away, the better. This number gives you an estimate of how many iterations it will take to equilibrate the current data. Keep in mind that this analysis only runs on the *already equilibrated data* and is therefor an esimate of how long it takes the system to relax in state and configuration space from this point.


Seeing something odd?
--------------------
* The diagonal is very dark, but everything else is white

You probably have poor mixing beteen states. This happens when there is insufficent phase space overlap between states and the probabilty of two replicas at different states swapping configurations approaches zero. If you have set the `mixing_warning_cutoff`, many of these states will be highlighted as warnings.

**Solution**: Add aditional states to your simulation near the states which are not mixing well. Provide a more gradual change of energy from the state to improve replica exchange from that state.

* Graph is mostly white!

This can happen if you have _too_ good of mixing alongside too many states. In this case, mixing between all states is happening so regularly that there is no concentration of configurations in one state.

**Solution**: Reduce `mixing_cutoff`.

* Its still way too white

That is a limitation of the custom colormap. You can try un-commenting the line `cmap = plt.get_cmap("Blues")` below to get a blue-scale colormap which has a far smaller white level so you can better see the diffusion in blue. You will lose the red warning color of states with too low a swap rate, but you can always comment the line back out to see those. The warning message will stil be generated.

**Solution**: Temporarily un-comment the line `cmap = plt.get_cmap("Blues")`

Options
-------
You can adust the `mixing_cutoff` options to control what threshold to display mixing. Anything below the cutoff will be shown as a blank. Defaults to `0.05`. Accepts either a float from `[0,1]` or `None` (`None` and `0` yeild the same result)

The `mixing_warning_threshold` is the level at which you determine there were insufficinet number of swaps between states. Consider adding additional states between the warnings and adjacent states to improve mixing. Accepts a float between `(mixing_cutoff,1]` (must be larger than `mixing_cutoff`). Defaults to `0.9` but this should be tuned based on the nubmer of states.

In [ ]:
# Set up image
from matplotlib.colors import LinearSegmentedColormap, NoNorm
plt.clf()
f, subplots = plt.subplots(1,2)
# Create custom cmap goes from white to pure blue, goes red if the threshold is reached
if mixing_cutoff is None:
    mixing_cutoff = 0
if mixing_warning_threshold <= mixing_cutoff:
    raise ValueError("mixing_warning_threshold must be larger than mixing_cutoff")
if mixing_warning_threshold > 1 or mixing_cutoff > 1 or mixing_warning_threshold < 0 or mixing_cutoff < 0:
    raise ValueError("mixing_warning_threshold and mixing_cutoff must be between [0,1]")
cdict = {'red':   ((0.0,  1.0, 1.0),
                   (mixing_cutoff, 1.0, 1.0),
                   (mixing_warning_threshold, 0.0, 0.0),
                   (mixing_warning_threshold, 1.0, 1.0),
                   (1.0,  1.0, 1.0)),

         'green': ((0.0,  1.0, 1.0),
                   (mixing_cutoff, 1.0, 1.0),
                   (mixing_warning_threshold, 0.0, 0.0),
                   (1.0,  0.0, 0.0)),

         'blue':  ((0.0,  1.0, 1.0),
                   (mixing_cutoff, 1.0, 1.0),
                   (mixing_warning_threshold, 1.0, 1.0),
                   (mixing_warning_threshold, 0.0, 0.0),
                   (1.0,  0.0, 0.0))}
cmap = LinearSegmentedColormap('BlueWarnRed', cdict)
# cmap = plt.get_cmap("Blues") # Use this cmap instead if your results are too diffuse to see over the white
for phase, subplot in zip(phases, subplots):
    mixing_data, mu = analyze.generate_mixing_statistics(ncfiles[phase], nequil=nequils[phase])
    # Without NoNorm, imgshow normalizes the values to mixing_data.max which can screw up the warning colormap
    output_image = subplot.imshow(mixing_data, aspect='equal', cmap=cmap, vmin=0, vmax=1)
    # Add colorbr
    decimal = 2 # Precision setting
    nticks = 11
    # The color bar has to be configured indepenent of the source image or it cant be truncated to only show the data
    # i.e. it would instead go 0-1 always
    ubound = np.min([np.around(mixing_data.max(),decimals=decimal) + 10**(-decimal), 1])
    lbound = np.max([np.around(mixing_data.min(),decimals=decimal) - 10**(-decimal), 0])
    boundslice = np.linspace(lbound,ubound,256)
    cbar = plt.colorbar(output_image, ax=subplot, orientation='vertical',
                        boundaries=boundslice,
                        values = boundslice[1:],
                        format='%.{}f'.format(decimal))
    # Update ticks
    ticks = np.around(np.linspace(lbound,ubound,nticks), decimals=decimal)
    ticks = np.linspace(lbound,ubound,nticks)
    cbar.set_ticks(ticks)
    # Labels
    title_txt = phase + " phase" + "\n" 
    title_txt += "Perron eigenvalue {0:9.5f}\nState equilibration timescale ~ {1:.1f} iterations".format(
            mu[1], 1.0 / (1.0 - mu[1]))
    subplot.set_title(title_txt, fontsize=20, y=1.05)
    # Display Warning
    if np.any(mixing_data >= mixing_warning_threshold):
        subplot.text(
            0.5, -0.2,
            "Warning!\nThere were states that less than {}% swaps!\nConsider adding more states!".format((1-mixing_warning_threshold)*100),
            verticalalignment='bottom', horizontalalignment='center',
            transform=subplot.transAxes,
            fontsize=20,
            color='red',
            bbox={'alpha':1.0, 'facecolor':'white', 'lw':0, 'pad':0}
            )
plt.show()

Still working on stuff below this point
=====================

Replica Pseudorandom Walk Examination
====================

This section checks to see if all the replicas are exchanging states over the whole thermodynamic state space. This is different from tracking states as any replica is a continuous trajectory of configurations, just undergoing different forces at different times.

What do I want to see here?
-------------------------

Each plot is its own replica, the line in each plot shows which *state* a given replica is in at time. The ideal scenario is that all replicas visit all states numerous times. If you see a line that is relatively flat, then you can surmise that very little mixing is occurring from that replica and you may wish to consider adding more states around the stuck region to "un-stick" it.

Something seem odd?
------------------
* All I see is black with some white dots mixed in (uncommon)

This is a good thing! It means the replicas are well mixed and are rapidly changing states. There may be some phases which were redundant though, which is not necessarily a bad thing since it just adds more samples at the given state, but it may mean you did extra work. An example of this is *decoupling* the steric forces of a ligand once *electrostatics have been annihilated* in *implicit* solvent. Since there is no change to the intra-molecular interactions at this point and the most solvent models are based on partial charges (which are now 0), all changes to the sterics are the same state.

* All my replicas stayed in the same state

A sign of very poor mixing. Consider adding additional states (see the **Mixing Statistics** section above for ideas on where). There may be other factors such as a low number of attempted replica swaps between each iteration.


In [ ]:
from matplotlib import gridspec
#Create Parent Gridspec
if phase_stacked_replica_plots:
    plot_grid = gridspec.GridSpec(2,1)
    plt.rcParams['figure.figsize'] = 20, 8*6
else:
    plot_grid = gridspec.GridSpec(1,2)
    plt.rcParams['figure.figsize'] = 20, 8*3
f = plt.figure()
for i, phase in enumerate(phases):
    # Gather state NK
    state_nk = ncfiles[phase].variables['states'][:,:]
    N, K = state_nk.shape
    # Create subgrid
    sub_grid = gridspec.GridSpecFromSubplotSpec(K, 1, subplot_spec=plot_grid[i])
    # Loop through all states
    for k in range(K):
        # Add plot
        plot = f.add_subplot(sub_grid[k])
        # Actually plot
        plot.plot(state_nk[:,k],'k.')
        # Format plot
        plot.set_yticks([])
        plot.set_xlim([0,N])
        plot.set_ylim([0,K])
        if k < K-1:
            plot.set_xticks([])
        plot.set_ylabel('{}'.format(k))
        if k == 0: # Title
            plot.set_title('{} phase'.format(phase), fontsize=20)

Free Energy Difference
============

The free energy difference is shown last as the quality of this estimate should be gauged with the eariler sections. Although MBAR provides an estimate of the free energy difference and its error, it is still only an estimate. You should consider if you have a sufficient number of decorelated samples, sufficient mixing/phase space overlap between states, and sufficient replica randomwalk to gauge the quality of this estimate.

In [ ]:
from simtk import unit as units
kB = units.BOLTZMANN_CONSTANT_kB * units.AVOGADRO_CONSTANT_NA

data = dict()
for phase in phases:
    ncfile = ncfiles[phase]
    DeltaF_restraints = 0.0
    if 'metadata' in ncfile.groups:
        # Read phase direction and standard state correction free energy.
        # Yank sets correction to 0 if there are no restraints
        DeltaF_restraints = ncfile.groups['metadata'].variables['standard_state_correction'][0]
        
    # Extract Energies
    (u_kln, N_k, u_n) = analyze.extract_ncfile_energies(ncfile, ndiscard=nequils[phase], g=g_ts[phase])
    
    # Create MBAR object to use for free energy and entropy states
    mbar = analyze.initialize_MBAR(ncfile, u_kln=u_kln, N_k=N_k)
    
    # Estimate free energies, use fully interacting state if present
    (Deltaf_ij, dDeltaf_ij) = analyze.estimate_free_energies(ncfile, mbar=mbar)

    # Estimate average enthalpies
    (DeltaH_i, dDeltaH_i) = analyze.estimate_enthalpies(ncfile, mbar=mbar)

    # Accumulate free energy differences
    entry = dict()
    entry['DeltaF'] = Deltaf_ij[0, -1]
    entry['dDeltaF'] = dDeltaf_ij[0, -1]
    entry['DeltaH'] = DeltaH_i[0, -1]
    entry['dDeltaH'] = dDeltaH_i[0, -1]
    entry['DeltaF_restraints'] = DeltaF_restraints
    data[phase] = entry

    # Get temperatures.
    ncvar = ncfile.groups['thermodynamic_states'].variables['temperatures']
    temperature = ncvar[0] * units.kelvin
    kT = kB * temperature
        
# Compute free energy and enthalpy
DeltaF = 0.0
dDeltaF = 0.0
DeltaH = 0.0
dDeltaH = 0.0
for phase, sign in analysis:
    DeltaF -= sign * (data[phase]['DeltaF'] + data[phase]['DeltaF_restraints'])
    dDeltaF += data[phase]['dDeltaF']**2
    DeltaH -= sign * (data[phase]['DeltaH'] + data[phase]['DeltaF_restraints'])
    dDeltaH += data[phase]['dDeltaH']**2
dDeltaF = np.sqrt(dDeltaF)
dDeltaH = np.sqrt(dDeltaH)

# Attempt to guess type of calculation
calculation_type = ''
for phase in phases:
    if 'complex' in phase:
        calculation_type = ' of binding'
    elif 'solvent1' in phase:
        calculation_type = ' of solvation'

print("Free energy{}: {:16.3f} +- {:.3f} kT ({:16.3f} +- {:.3f} kcal/mol)".format(
    calculation_type, DeltaF, dDeltaF, DeltaF * kT / units.kilocalories_per_mole,
    dDeltaF * kT / units.kilocalories_per_mole))

for phase in phases:
    print("DeltaG {:<25} : {:16.3f} +- {:.3f} kT".format(phase, data[phase]['DeltaF'],
                                                        data[phase]['dDeltaF']))
    if data[phase]['DeltaF_restraints'] != 0.0:
        print("DeltaG {:<25} : {:25.3f} kT".format('restraint', data[phase]['DeltaF_restraints']))
print('')
print("Enthalpy{}: {:16.3f} +- {:.3f} kT ({:16.3f} +- {:.3f} kcal/mol)".format(
    calculation_type, DeltaH, dDeltaH, DeltaH * kT / units.kilocalories_per_mole,
    dDeltaH * kT / units.kilocalories_per_mole))

----
Primers
====

Equilibration Primer
===========

Is equilibration necessary?
---------------------------

In principle, we don't need to discard initial "unequilibrated" data; the estimate over a very long trajectory will converge to the correct free energy estimate no matter what---we simply need to run long enough.  Some MCMC practitioners, like Geyer, feel strongly enough about this to throw up a webpage in defense of this position:

http://users.stat.umn.edu/~geyer/mcmc/burn.html

In practice, if the initial conditions are very atypical of equilibrium (which is often the case in molecular simulation), it helps a great deal to discard an initial part of the simulation to equilibration.  But how much?  How do we decide?

Determining equilibration in a replica-exchange simulation
----------------------------------------------------------

For a standard molecular dynamics simulation producing a trajectory $x_t$, it's reasonably straightforward to decide approximately how much to discard if human intervention is allowed.  We simply look at some property $A_t = A(x_t)$ over the course of the simulation---ideally, a property that we know has some slow behavior that may affect the quantites we are intersted in computing ($A(x)$ is a good choice if we're interested in the expectation $<A>$) and find the point where $A_t$ seems to have "settled in" to typical equilibrium behavior.

If we're interested in a free energy, which is computed from the potential energy differences, let's suppose the potential energy $U(x)$ may be a good quantity to examine.

But in a replica-exchange simulation, there are K replicas that execute nonphysical walks on many potential energy functions $U_k(x)$.  What quantity do we look at here?

Let's work by analogy.  In a single simulation, we would plot some quantity related to the potential energy $U(x)$, or its reduced version $u(x) = \beta U(x)$.  This is actually the negative logarithm of the probability density $\pi(x)$ sampled, up to an additive constant:

$$\pi(x) = Z^{-1} e^{-u(x)}$$
$$u(x) = -\ln \pi(x) + c$$

For a replica-exchange simulation, the sampler state is given by the pair $(X,S)$, where $X = \{x_1, x_2, \ldots, x_K \}$ are the replica configurations and $S = \{s_1, s_2, \ldots, s_K\}$ is the vector of state index permutations associated with the replicas.  The total probability sampled is

$$\Pi(X,S) = \prod_{k=1}^K \pi_{s_k}(x_k) = (Z_1 \cdots Z_K) \exp\left[-\sum_{k=1}^K u_{s_k}(x_k)\right] = Q^{-1} e^{-u_*(X)}$$

where the pseudoenergy $u_*(X)$ for the replica-exchange simulation is defined as

$$u_*(X) \equiv \sum_{k=1}^K u_{s_k}(x_k)$$

That is, $u_*(X)$ is the sum of the reduced potential energies of each replica configuration at the current thermodynamic state it is vsiting.

Let's look at the timeseries for this quantity:

Mixing Statistics Primer
=============

How we compute the mixing ratios
------------------------------

In practice, this is done by recording the number of times a replica transitions from alchemical state $i$ to state $j$ in a single iteration.  Because the overall chain must obey detailed balance, we count each transition as contributing 0.5 counts toward the $i \rightarrow j$ direction and 0.5 toward the $j \rightarrow i$ direction.  This has the advantage of ensuring that the eigenvalues of the resulting transition matrix among alchemical states are purely real.

Interpreting the Perron (subdominant/second) Eigenvalue 
----------------------------------------------------

If the subdominant eigenvalue would have been unity, then the chain would be *decomposable*, meaning that it completely separated into two separate sets of alchemical states that did not mix.  This would have been an indication of poor phase space overlap between some alchemical states.

In practice, it's a great idea to monitor these statistics as the simulation is running, even if no data is discarded to equilibration at that point.  They give not only a good idea of whether sufficient mixing is occuring, but it provides a lower bound on the mixing time in configuration space.

If the configuration $x$ sampling is infinitely fast so that $x$ can be considered to be at equilibrium given the instantaneous permutation $S$ of alchemical state assignments, the subdominant eigenvalue $\lambda_2 \in [0,1]$ gives an estimate of the mixing time of the overall $(X,S)$ chain:
    
$$\tau = \frac{1}{1 - \lambda_2}$$

Now, in most cases, the configuration $x$ sampling is *not* infinitely fast, but at least we can use $\tau$ to get a very crude estimate of how quickly each replica relaxes in $(X,S)$ space.

Gelman-Rubin Convergence Primer
====================================

In 1992, Gelman and Rubin proposed a very clever idea for a convergence diagnostic in the case that multiple MCMC samplers are run from different initial sampler states:

http://dx.doi.org/10.1214/ss/1177011136

The idea is simple: Each chain gives an individual estimate for some computed expectation or property, and the whole collection of chains give a (presumably more precise) estimate.  We can simply compare the individual estimates to the overall estimate to determine whether the chains have been run long enough to see concordance between the individual and global estimates, to within appropriate statistical error.  If not, then the samplers have not yet run long enough to sample all of the important modes of the density.
    
We can apply a similar idea here, especially if we have initialized our replicas with different configurations (e.g. different docked ligand conformations, and potentially different protein conformations as well).

**Note**: This feature has not yet been added